# DMA tutorial: DMA to stream interfaces

This overlay consists of two DMAs and an AXI Stream FIFO (input and output AXI stream interfaces). The FIFO represents an accelerator. A single DMA could be used with a read and write channel enabled, but for demonstration purposes, two different DMAs will be used. 

* The first DMA with read channel enabled is connected from DDR to IP input stream (reading from DDR, and sending to AXI stream).
* The second DMA has a write channel enabled and is connected to IP output stream to DDR (receiving from AXI stream, and writing to DDR memory).


![](images/dma_stream_example.png)

## 1. Downloading overlay
The overlay can be downloaded automatically when instantiating an overlay class.

In [18]:
from pynq import Overlay

In [19]:
overlay = Overlay("./bitstream/dma_tutorial.bit")

We can check the IPs in this overlay. Notice the DMAs *axi_dma_from_pl_to_ps* and *axi_dma_from_pl_to_ps*.

In [20]:
overlay.ip_dict

{'axi_dma_from_pl_to_ps': {'addr_range': 65536,
  'driver': pynq.lib.dma.DMA,
  'fullpath': 'axi_dma_from_pl_to_ps',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S_AXI_LITE',
  'parameters': {'C_BASEADDR': '0x40400000',
   'C_DLYTMR_RESOLUTION': '125',
   'C_ENABLE_MULTI_CHANNEL': '0',
   'C_FAMILY': 'zynq',
   'C_HIGHADDR': '0x4040FFFF',
   'C_INCLUDE_MM2S': '0',
   'C_INCLUDE_MM2S_DRE': '0',
   'C_INCLUDE_MM2S_SF': '1',
   'C_INCLUDE_S2MM': '1',
   'C_INCLUDE_S2MM_DRE': '0',
   'C_INCLUDE_S2MM_SF': '1',
   'C_INCLUDE_SG': '0',
   'C_INCREASE_THROUGHPUT': '0',
   'C_MICRO_DMA': '0',
   'C_MM2S_BURST_SIZE': '16',
   'C_M_AXIS_MM2S_CNTRL_TDATA_WIDTH': '32',
   'C_M_AXIS_MM2S_TDATA_WIDTH': '32',
   'C_M_AXI_MM2S_ADDR_WIDTH': '32',
   'C_M_AXI_MM2S_DATA_WIDTH': '32',
   'C_M_AXI_S2MM_ADDR_WIDTH': '32',
   'C_M_AXI_S2MM_DATA_WIDTH': '32',
   'C_M_AXI_SG_ADDR_WIDTH': '32',
   'C_M_AXI_SG_DATA_WIDTH': '32',
   'C_NUM_MM2S_CHANNELS': '1',
   'C_NUM_S2MM_CHANNELS': '1',
   'C_PRMRY_IS_ACLK_A

## 2. Create DMA instances

Using the labels for the DMAs listed above, we can create two DMA objects.

In [21]:
import pynq.lib.dma

dma_send = overlay.axi_dma_from_ps_to_pl
dma_recv = overlay.axi_dma_from_pl_to_ps

## 3. Read DMA
We will read some data from memory, and write to FIFO in the following cells.

The first step is to allocate the buffer. pynq.allocate will be used to allocate the buffer, and NumPy will be used to specify the type of the buffer. 

In [22]:
# from pynq import allocate
from pynq import Xlnk
import numpy as np

xlnk = Xlnk()
data_size = 100
input_buffer = xlnk.cma_array(shape=(data_size,), dtype=np.uint32)

The array can be used like any other NumPy array. We can write some test data to the array. Later the data will be transferred by the DMA to the FIFO. 

In [23]:
for i in range(data_size):
    input_buffer[i] = i + 0xcafe0000

Let's check the contents of the array. The data in the following cell will be sent from PS (DDR memory) to PL (streaming FIFO).

### Print first few values of buffer 

In [24]:
for i in range(10):
    print(hex(input_buffer[i]))

0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


Now we are ready to carry out DMA transfer from a memory block in DDR to FIFO.

In [25]:
dma_send.sendchannel.transfer(input_buffer)

## 4. Write DMA
Let's read the data back from FIFO stream, and write to MM memory. The steps are similar.

We will prepare an empty array before reading data back from FIFO.

### Print first few values of buffer 

In [12]:
# output_buffer = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer = xlnk.cma_array(shape=(data_size,), dtype=np.uint32)

for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


In [13]:
dma_recv.recvchannel.transfer(output_buffer)

The next cell will print out the data received from PL (streaming FIFO) to PS (DDR memory). This should be the same as the data we sent previously.

### Print first few values of buffer 

In [14]:
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


Verify that the arrays are equal

In [15]:
print("Array are equal: {}".format(np.array_equal(input_buffer, output_buffer)))

Array are equal: True


## 6. Free all the memory buffers
Don't forget to free the memory buffers to avoid memory leaks!

In [16]:
del input_buffer, output_buffer

## Appendix

## AXI DMA pdf

PDFs can be linked from, or embedded in a Jupyter notebook. The pdf for the AXI DMA documentation is embedded in this notebook below:

In [17]:
from IPython.display import IFrame
IFrame("pg021_axi_dma.pdf", width=800, height=800)